In [5]:
!pip install nltk

In [7]:
!pip install tflearn

  Stored in directory: C:\Users\Nata\AppData\Local\pip\Cache\wheels\d0\f6\69\0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [8]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np

import tensorflow as tf
import random
import tflearn




curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [0]:
#tflearn.__version__


AttributeError: ignored

In [9]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nata\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [0]:
drive.mount("./drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./drive


In [0]:
intents = []

In [13]:
!cp prueba4.json .


cp: 'prueba4.json' and './prueba4.json' are the same file


In [11]:
!ls

chatbotv2.ipynb
prueba3.json
prueba4.json


In [15]:
# import our chat-bot intents file
import json
with open('prueba4.json') as json_data:
    intents = json.load(json_data)

In [16]:
intents

{'intents': [{'tag': 'saludo',
   'patterns': ['Hola',
    'holii',
    'Buenos dias',
    'Buenas tardes',
    'Buenas noches',
    'wenas'],
   'responses': ['Hola, gracias por reunirse conmigo. Â¿me podria indicar con quien estoy hablando?',
    'Buenas, gracias por esta reunion. Â¿me podria indicar con quien estoy hablando?'],
   'context_set': ''},
  {'tag': 'nomre',
   'patterns': ['yo soy', 'mi nombre es', 'me llamo'],
   'responses': ['Â¿que cargo desempeÃ±a?',
    'Â¿cual es el cargo que desempeÃ±a dentro de la compaÃ±ia?'],
   'context_set': 'cargo'},
  {'tag': 'cargo',
   'patterns': ['yo soy el', 'yo soy la', 'yo trabajo de', 'mi cargo es'],
   'responses': ['Â¿Cuales son sus principales responsabilidades?',
    'Â¿que funciones desempeÃ±a?'],
   'context_filter': 'cargo'},
  {'tag': 'responsabilidades',
   'patterns': ['yo me encargo de',
    'entre mis responsabilidades esta',
    'lo que debo hacer es'],
   'responses': ['Â¿que entregables produce?',
    'Â¿cuales son su

In [19]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

20 documents
5 classes ['cargo', 'entregables', 'nomre', 'responsabilidades', 'saludo']
33 unique stemmed words ['buena', 'bueno', 'cargo', 'de', 'debo', 'dia', 'el', 'encargo', 'ent', 'entreg', 'es', 'est', 'hac', 'hol', 'holi', 'la', 'llamo', 'lo', 'los', 'me', 'mi', 'mis', 'noch', 'nombr', 'que', 'responsabilidad', 'son', 'soy', 'tard', 'trabajo', 'un', 'wena', 'yo']


In [20]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [21]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')


Training Step: 2999  | total loss: 0.46136 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.46136 - acc: 0.9324 -- iter: 16/20
Training Step: 3000  | total loss: 0.86469 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.86469 - acc: 0.8642 -- iter: 20/20
--
INFO:tensorflow:C:\Users\Nata\Desktop\deep learning\npl\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [22]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )


In [23]:
!ls

chatbotv2.ipynb
checkpoint
model.tflearn.data-00000-of-00001
model.tflearn.index
model.tflearn.meta
prueba3.json
prueba4.json
tflearn_logs
training_data


In [27]:

# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']



# import our chat-bot intents file
import json
with open('prueba4.json') as json_data:
    intents = json.load(json_data)

In [28]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Nata\Desktop\deep learning\npl\chatbot\model.tflearn


In [29]:

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [32]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [33]:
response('hola')

Hola, gracias por reunirse conmigo. Â¿me podria indicar con quien estoy hablando?


In [0]:
classify('quisiera comprar algo, pero no se que')

[(122, 0.98035264)]

In [0]:
response('moped')

Muchas gracias por visitarnos


In [34]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

nombres = []
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']
                        


                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        nombres = [sentence]
                        nombre = nombres[0]
                        # a random response from the intent
                        x = random.choice(i['responses']).format(nombre)
                        return x

            results.pop(0)


In [46]:

response('me encargo de programar')

'Â¿que entregables produce?'